In [ ]:

import simple_train

transform=simple_train.get_transform(train=True)
transform

In [ ]:

dataset = simple_train.PennFudanDataset('PennFudanPed', transform)
dataset[0]


In [ ]:
import simple_train
model=simple_train.get_fasterrcnn_model_instance_segmentation(2)
print(model.rpn)

In [ ]:
import torchvision
from torchvision.models.detection.rpn import AnchorGenerator
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import torch
def get_rcnn():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=False,
        image_mean=[0.485, 0.456, 0.406, 0.485, 0.456, 0.406],
        image_std=[0.229, 0.224, 0.225, 0.229, 0.224, 0.225])
    return model

def get_custom_rcnn(input_channels=6,num_classes=2):
    #anchor_sizes = ((2), (4), (8), (16), (32)) 
    anchor_sizes = ((32), (64), (128), (256), (512))
    aspect_ratios = ((0.5, 1.0, 1.5, 3), (0.5, 1.0, 1.5, 3),
                     (0.5, 1.0, 1.5, 3), (0.5, 1.0, 1.5, 3),
                     (0.5, 1.0, 1.5, 3))
    anchor_generator = AnchorGenerator(
        sizes=anchor_sizes, aspect_ratios=aspect_ratios)

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(
        pretrained=False,
        image_mean=[0.485, 0.456, 0.406, 0.485, 0.456, 0.406],
        image_std=[0.229, 0.224, 0.225, 0.229, 0.224, 0.225],
        #rpn_anchor_generator=anchor_generator,
        trainable_backbone_layers=5)
    in_features = model.roi_heads.box_predictor.cls_score.in_features
    # replace the pre-trained head with a new one
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)


    # original
    # (box_predictor): FastRCNNPredictor(
    #    (cls_score): Linear(in_features=1024, out_features=91, bias=True)
    #    (bbox_pred): Linear(in_features=1024, out_features=364, bias=True)
    #)
    #(box_predictor): FastRCNNPredictor(
    #    (cls_score): Linear(in_features=1024, out_features=2, bias=True)
    #    (bbox_pred): Linear(in_features=1024, out_features=8, bias=True)
    #)

    model.backbone.body.conv1 = torch.nn.Conv2d(input_channels, 64, kernel_size=7,
                                                stride=2, padding=3, bias=False)
    return model

model=get_custom_rcnn()

print(model.rpn.anchor_generator.sizes)
print(model.rpn.anchor_generator.aspect_ratios)
print(model.rpn.anchor_generator.num_anchors_per_location())
print(model.rpn.anchor_generator.cell_anchors)
print(f"{model.rpn_score_thresh}")



In [ ]:

for name, param in model.named_parameters():
    print(f"{name}:{param.requires_grad}")
print(model)


In [ ]:
for name, parameter in model.named_parameters():
    print(f"{name}, {parameter.size()}")


In [ ]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad:
            continue
        param = parameter.numel()
        table.add_row([name, param])
        total_params += param
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params


count_parameters(model)


In [ ]:
import torch
#for name, layer in model.named_parameters():
#    print(f"{name}, {layer.size()}")
#    model[name] = torch.nn.Conv2d(4, 64, kernel_size=(7, 7), stride=(
#        2, 2), padding=(3, 3), bias=False)
model.backbone.body.conv1 = torch.nn.Conv2d(4, 64, kernel_size=7,
                        stride=2, padding=3, bias=False)

print(model)


In [ ]:
import torch
from engine import train_one_epoch, evaluate
import simple_train
import os
import utils

device = torch.device(
    'cuda') if torch.cuda.is_available() else torch.device('cpu')

dataset_test = simple_train.PesmodOpticalFlowDataset(
    os.path.join('PESMOD', 'train'), simple_train.get_transform(train=False)) # !!actually training data
indices = torch.randperm(len(dataset_test)).tolist()
dataset_test = torch.utils.data.Subset(dataset_test, indices[:50])
print(f"test set size: {len(dataset_test)}")

data_loader_test = torch.utils.data.DataLoader(
    dataset_test, batch_size=1, shuffle=False, num_workers=4,
    collate_fn=utils.collate_fn)

model = simple_train.get_fasterrcnn_model(input_channels=6, classes=2)

# move model to the right device
model.to(device)


evaluate(model, data_loader_test, device=device)


